## 1. Set up the environment


In [5]:
!pip install myosuite==2.8.3 --quiet
!pip install "stable-baselines3[extra]" --quiet
!pip install tqdm --quiet
!pip install mujoco==3.1.2 --quiet
!pip install sk-video --quiet
%env MUJOCO_GL=egl
import mujoco

env: MUJOCO_GL=egl


In [6]:
import myosuite
from myosuite.utils import gym
import skvideo.io
import numpy as np
import os
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import CheckpointCallback
# from tqdm import tqdm_notebook as tqdm
from tqdm.notebook import tqdm
from IPython.display import HTML
from base64 import b64encode
# !pip install wget
# import wget
import envs

def show_video(video_path, video_width = 400):

  video_file = open(video_path, "r+b").read()

  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video autoplay width={video_width} controls><source src="{video_url}"></video>""")

## 2. Prepare env and model


In [7]:
env_name = 'CustomBimanualEnv-v0'
env = gym.make(env_name)

/home/ta747375ki/myochallenge-neuroflex/envs/bimanual.py:488: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.manip_joint_range = np.arange(model.joint("manip_object/freejoint").qposadr,
/home/ta747375ki/myochallenge-neuroflex/envs/bimanual.py:488: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.manip_joint_range = np.arange(model.joint("manip_object/freejoint").qposadr,
/home/ta747375ki/myochallenge-neuroflex/envs/bimanual.py:488: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Depreca

In [8]:
checkpoint_callback = CheckpointCallback(save_freq=1000, save_path='./logs/', name_prefix='ppo_model')
# model = PPO.load('baseline', env)
# model = PPO.load("./logs/ppo_model_6000_steps.zip", env)
model = PPO.load("./agent/model.zip", env, device='cpu')  # Load model on CPU
env.reset()
env.seed(51) #setting a seed for the env
tb_log_name = "ppo_training"
# model.learn(total_timesteps=5000, callback=checkpoint_callback, tb_log_name=tb_log_name)

In [10]:
# Ignore specific warning
import warnings
from datetime import datetime
warnings.filterwarnings("ignore", message=".*tostring.*is deprecated.*")

movie = True #we want to save a video
frames = []
view = 'front'
for _ in range(3):
    ep_rewards = []
    done = False
    obs = env.reset()
    for step in range(400):
        # if done: break
        obs = env.obsdict2obsvec(env.obs_dict, env.obs_keys)[1]
        # print(f"obs shape: {len(obs)}")
        # print(f"obs keys: {env.obs_keys}")
        action, _ = model.predict(obs, deterministic=True)
        # hard-coding the myoHand to release object
        action[30] = 1
        if step > 130:
    #   if obs.
            action[32:40] = 0
            action[40:49] = 1

        #hard coding the MPL to the desire position, since we know the actuation of the MPL is the last 17 index of action
        action[-17:] = np.array([-0.65001469 , 1.     ,    -0.23187843 , 0.59583695 , 0.92356688, -0.16,
                                -0.28 ,      -0.88   ,     0.25 ,      -0.846   ,   -0.24981132 ,-0.91823529,
                                -0.945  ,    -0.925   ,   -0.929   ,   -0.49    ,   -0.18      ])
        if step > 250:
            action[-17:] = np.array([-0.4199236 ,  1.      ,   -0.9840558 ,  0.35299219,  0.92356688,  0.02095238,
                                        -0.28    ,   -0.88  ,      0.25      , -0.846     , -0.24981132, -0.91823529,
                                        -0.945   ,   -0.925   ,   -0.929    ,  -0.49     ,  -0.918     ])
        if step > 350:
            action[-17:] = np.array([-0.4199236 ,  1.     ,    -0.9840558,   0.35299219 , 0.3910828 ,  0.02095238,
                                        -0.28    ,   -0.88     ,   0.25   ,    -0.846     , -0.24981132 ,-0.91823529,
                                        -0.945    ,  -0.925    ,  -0.929    ,  -0.49  ,     -0.918     ])

        obs, reward, done, info, info_2 = env.step(action)
        if movie:
            geom_1_indices = np.where(env.sim.model.geom_group == 1)
            env.sim.model.geom_rgba[geom_1_indices, 3] = 0
            frame = env.sim.renderer.render_offscreen(width= 400, height=400,camera_id=1)
            frames.append(frame)

env.close()

os.makedirs('videos', exist_ok=True)
# make a local copy
current_date = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
skvideo.io.vwrite(f'videos/test_policy_{current_date}.mp4', np.asarray(frames), inputdict = {'-r': '100'}, outputdict={"-pix_fmt": "yuv420p"})
show_video(f'videos/test_policy_{current_date}.mp4')

/home/ta747375ki/myochallenge-neuroflex/venv/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.obsdict2obsvec to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.obsdict2obsvec` for environment variables or `env.get_wrapper_attr('obsdict2obsvec')` that will search the reminding wrappers.
  logger.warn(
/home/ta747375ki/myochallenge-neuroflex/venv/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.obs_dict to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.obs_dict` for environment variables or `env.get_wrapper_attr('obs_dict')` that will search the reminding wrappers.
  logger.warn(
/home/ta747375ki/myochallenge-neuroflex/venv/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.obs_keys to get variables from other wrappers is deprecated and will be removed in v1.0, t

You will first notice that only in some occasion the myoHand is grasping the object. This is because the baseline is trained upon simple no variation conditions. To fully replicate the baseline behavior, initiate the env with no variations:


```
kwargs={
            'model_path': curr_dir + '/../assets/arm/myoarm_bionic_bimanual.xml',
            'normalize_act': True,
            'frame_skip': 5,
            'obj_scale_change': None
            'obj_mass_change': None
            'obj_friction_change': None
        }
```

!Note: The evaluation env will still have variations

## With all of those set, you can fully replicate the new baseline solution and build on top of it. Happy training!